In [8]:
from bs4 import BeautifulSoup
import requests as rq 
import dateparser
import sqlite3
from collections import namedtuple


In [2]:
OMDB_API_KEY = '76708f51'

In [8]:
resp = rq.get('http://www.omdbapi.com/?i=tt3896198&apikey=76708f51&tomatoes=true')

In [8]:
resp = rq.get('http://www.omdbapi.com/?i=nm0000198&apikey=76708f51')

In [11]:
resp.__dict__

{'_content': b'{"Response":"False","Error":"Error converting data type varchar to int."}',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Date': 'Sat, 26 Jan 2019 15:54:23 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=df2448e915542483a98339e6f31b086c61548518063; expires=Sun, 26-Jan-20 15:54:23 GMT; path=/; domain=.omdbapi.com; HttpOnly', 'Cache-Control': 'public, max-age=86400', 'Expires': 'Sun, 27 Jan 2019 15:54:23 GMT', 'Last-Modified': 'Sat, 26 Jan 2019 15:53:52 GMT', 'Vary': '*, Accept-Encoding', 'X-AspNet-Version': '4.0.30319', 'X-Powered-By': 'ASP.NET', 'Access-Control-Allow-Origin': '*', 'CF-Cache-Status': 'HIT', 'Server': 'cloudflare', 'CF-RAY': '49f4286ae1296afb-WAW', 'Content-Encoding': 'gzip'},
 'raw': <urllib3.response.HTTPResponse at 0x1102d7978>,
 'url': 'http://www.omdbapi.com/?i=nm0000198&apikey=76708f51',
 'encoding': 'utf-8',
 'history': [],
 

In [9]:
resp.json()

{'Title': 'Guardians of the Galaxy Vol. 2',
 'Year': '2017',
 'Rated': 'PG-13',
 'Released': '05 May 2017',
 'Runtime': '136 min',
 'Genre': 'Action, Adventure, Comedy, Sci-Fi',
 'Director': 'James Gunn',
 'Writer': 'James Gunn, Dan Abnett (based on the Marvel comics by), Andy Lanning (based on the Marvel comics by), Steve Englehart (Star-Lord created by), Steve Gan (Star-Lord created by), Jim Starlin (Gamora and Drax created by), Stan Lee (Groot created by), Larry Lieber (Groot created by), Jack Kirby (Groot created by), Bill Mantlo (Rocket Raccoon created by), Keith Giffen (Rocket Raccoon created by), Steve Gerber (Howard the Duck created by), Val Mayerik (Howard the Duck created by)',
 'Actors': 'Chris Pratt, Zoe Saldana, Dave Bautista, Vin Diesel',
 'Plot': "The Guardians must fight to keep their newfound family together as they unravel the mystery of Peter Quill's true parentage.",
 'Language': 'English',
 'Country': 'USA',
 'Awards': 'Nominated for 1 Oscar. Another 12 wins & 42 n

In [132]:
Review = namedtuple('Review', ['movie_url', 'reviewer_type', 'reviewer_name', 'reviewer_url', 'rating', 'text', 'date'])

In [133]:
TYPE_CRITIC = 'critic'
TYPE_USER = 'user'

In [48]:
# class that grabs Rotten Tomatoes scores and link to reviews page
class RTMovie:

    # initialized to website of movie on Rotten Tomatoes
    def __init__(self, url):
        self.reviewUrl = ''
        self.set = False
        self.url = url
        self.page = rq.get(url).text
        self.soup = BeautifulSoup(self.page, 'lxml')

    # get the url for the first page of movie reviews
    def getReviewUrl(self):
        if not self.set:
            self.buildReviewUrl()
        return self.reviewUrl

    # Tomatometer score
    def criticScore(self):
        meter = "meter-value superPageFontColor"
        score = self.soup.find('span', {'class': meter})

        return score.text

    def audienceScore(self):
        audience_meter = "audience-score meter"
        audience_score = "superPageFontColor"

        meter = self.soup.find('div', {'class': audience_meter})
        score = meter.find('span', {'class': audience_score})

        return score.text

    # helper function for getting the review url
    def buildReviewUrl(self, set=False):
        if self.url[-1] != '/':
            self.url = self.url + '/'
        self.reviewUrl = self.url + "reviews/?type=user"
        return

In [49]:
# Audience Reviews inherits from RTScore
class BaseReviews(RTScore):

    def __init__(self, movie_url):
        RTScore.__init__(self, url)
        self.url = url
        self.paging = 0
        self.curSoup = self.soup 
        self.nextPageUrl = url

    # to get the url of the next page
    def nextPage(self):
        paging = 'pageInfo'
        base_url = "https://www.rottentomatoes.com"
        page_section = self.curSoup.find('span', {'class': paging})
        next_page = ''
        # return page_section

        try:
            next_page = page_section.findNext('a', {'class': 'btn btn-xs btn-primary-rt'})['href']
        except:
            self.nextPageUrl = None
            return

        self.paging += 1
        self.nextPageUrl = base_url + next_page
        # print(self.nextPageUrl)
        return self.nextPageUrl

    def getReviews(self):
        raise NotImplemented

In [50]:
# Audience Reviews inherits from RTScore
class AudienceReviews(BaseReviews):
    def getReviews(self):
		reviews = self.curSoup.find('div', {'id': 'reviews'})
		cur = reviews.findNext('div', {'class': 'user_review'})

		review_list = []

		while cur:
			review_list.append(cur.text.strip())
			cur = cur.findNext('div', {'class': 'user_review'})
		return review_list

In [ ]:
# Audience Reviews inherits from RTScore
class CriticReviews(RTScore):
    def getReviews(self):
        review_list = []
        reviews = self.curSoup.find('div', {'id': 'reviews'})
        review = reviews.findNext('div', {'class': 'row review_table_row'})
        while review:
            critic = review.find('div', {'class': 'critic_name'}).find('a', {'class': 'articleLink'})
            critic_name = critic.text.strip()
            critic_url = critic.get('href')
            date = review.find('div', {'class': 'review_date'}).text.strip()
            text = review.find('div', {'class': 'the_review'}).text.strip()
            rating = int(bool(review.find('div', {'class': 'fresh'})))
            review_list.append(Review(self.movie_url, TYPE_CRITIC, critic_name, critic_url, rating, text, date))
            review = review.findNext('div', {'class': 'row review_table_row'})
        return review_list

In [8]:
class Reviews():
    

	def __init__(self, movie_url, limit=10):
        if rev_type == 'user':
            url += '?type=user'
            self.page = AudienceReviews(url)
        elif rev_type == 'critic':    
            self.page = CriticReviews(url)
        else:
            raise Exception("Unknown rev_type: %s" % rev_type)
		self.limit = limit
		self.reviews = []


	# helper function to page for reviews, outputs as an array of reviews
	def getReviews(self):
		for i in range(self.limit):
			self.reviews += self.page.getReviews()
			next_page = self.page.nextPage()

			if next_page:
				self.page = AudienceReviews(next_page)
			else:
				break

		# return ''.join(self.reviews)
		return self.reviews

In [40]:
page = RTScore('https://www.rottentomatoes.com/m/cold_war_2018/')
critic_score = page.criticScore()
audience_score = page.audienceScore()
cw_reviews = TextReviews(page.getReviewUrl(), limit=10)

In [37]:
reviews = cw_reviews.getReviews()

In [36]:
soup = BeautifulSoup(rq.get('https://www.rottentomatoes.com/m/cold_war_2018/reviews/?page=9').text, 'lxml')

In [37]:
page_section = soup.find('span', {'class': 'pageInfo'})

In [39]:
next_page = page_section.findNext('a', {'class': 'btn btn-xs btn-primary-rt'})

In [40]:
next_page

<a class="btn btn-xs btn-primary-rt" href="/m/cold_war_2018/reviews/?page=8&amp;sort="><span class="glyphicon glyphicon-chevron-left"></span></a>

In [42]:
r = rq.get('https://www.rottentomatoes.com/m/cold_war_2018/reviews/?page=10')

In [44]:
r.ok

False

# Critic Reviews

In [2]:
page_text = rq.get('https://www.rottentomatoes.com/m/cold_war_2018/reviews/').text

In [3]:
soup = BeautifulSoup(page_text, 'lxml')

In [5]:
reviews = soup.find('div', {'id': 'reviews'})

In [55]:
reviews

<div id="reviews"> <ul class="nav nav-tabs hidden-xs"> <li class="active"><a href="/m/cold_war_2018/reviews/">All Critics</a></li> <li><a href="/m/cold_war_2018/reviews/?type=top_critics">Top Critics</a></li> <li><a href="/m/cold_war_2018/reviews/?type=my_critics">My Critics</a></li> <li><a href="/m/cold_war_2018/reviews/?type=user">Audience</a></li> </ul> <div class="visible-xs btn-group review_title"> <button aria-expanded="false" class="dropdown-toggle btn btn-default" data-toggle="dropdown" type="button"> All Critics<span class="fontelloIcon icon-down-dir"></span> </button> <ul class="dropdown-menu" role="menu"> <li class="active"><a href="/m/cold_war_2018/reviews/">All Critics</a></li> <li><a href="/m/cold_war_2018/reviews/?type=top_critics">Top Critics</a></li> <li><a href="/m/cold_war_2018/reviews/?type=my_critics">My Critics</a></li> <li><a href="/m/cold_war_2018/reviews/?type=user">Audience</a></li> </ul> </div> <div class="content"> <div style="display:inline-block; float:rig

In [6]:
review_children = reviews.findChildren('div', {'class': 'row review_table_row'})

In [34]:
for i, review in enumerate(review_children):
    critic = review.find('div', {'class': 'critic_name'})
    if not critic:
        print(f"\n\nNo Critic in {i}")
        continue
    critic_link = critic.find('a', {'class': 'articleLink'})
    if not critic_link:
        print(f"\n\nNo link in {i}!")
        continue
    critic_name = critic_link.text.strip()
    critic_url = critic_link.get('href')
    rdate = review.find('div', {'class': 'review_date'})
    if not rdate:
        print(f'\n\nNo date in {i}!')
        continue
    date = dateparser.parse(rdate.text.strip()).date()
    text = review.find('div', {'class': 'the_review'}).text.strip()
    fresh = bool(review.find('div', {'class': 'fresh'}))
    small_subtle = review.find('div', {'class': 'small subtle'})
    if small_subtle:
        score = re.search("Original Score: (.+)$", small_subtle.text)
        if score:
            score_text = score.group(1)
        else:
            score_text = None
    print(i, critic_name, critic_url, date, rating, score_text)

0 Sergio Huidobro /critic/sergio-huidobro/ 2019-01-27 1 5/5
1 Harvey S. Karten /critic/harvey-s-karten/ 2019-01-27 1 B
2 Sarah Gopaul /critic/sarah-gopaul/ 2019-01-25 1 None
3 Frank Swietek /critic/frank-swietek/ 2019-01-25 1 A-
4 Anne Brodie /critic/anne-brodie/ 2019-01-25 1 None
5 Chris Knight /critic/chris-knight/ 2019-01-25 1 4.5/5
6 Matthew Lickona /critic/matthew-lickona/ 2019-01-25 1 3/5
7 Peter Rainer /critic/peter-rainer/ 2019-01-25 1 B+
8 Max Weiss /critic/max-weiss/ 2019-01-25 1 3/4
9 Jeffrey Edalatpour /critic/jeffrey-edalatpour/ 2019-01-24 1 None
10 Robert Denerstein /critic/robert-denerstein/ 2019-01-24 1 None
11 Beth Accomando /critic/beth-accomando/ 2019-01-24 1 None
12 Hannah Brown /critic/hannah-brown/ 2019-01-24 1 None
13 Marjorie Baumgarten /critic/marjorie-baumgarten/ 2019-01-24 1 3/5
14 Ken Eisner /critic/ken-eisner/ 2019-01-24 1 None
15 Peter Howell /critic/peter-howell/ 2019-01-24 1 3.5/4
16 Peter Canavese /critic/peter-canavese/ 2019-01-23 1 3.5/4
17 Joe Friar 

In [18]:
t = review.find('div', {'class': 'small subtle'}).text

In [19]:
import re

In [32]:
re.match("Original Score: (.+)$", "Original Score: 3.4/10").group(1)

'3.4/10'

In [33]:
re.match?

In [15]:
review

<div class="row review_table_row"> <div class="col-xs-8"> <div class="col-sm-7 col-xs-16 critic_img"> <img class="critic_thumb fullWidth" src="https://resizing.flixster.com/GKyvv4VCdOe4ub8J8SeRXrx0WfQ=/72x72/v1.YzsyODAwO2c7MTc5NjU7MjA0ODsxNTA7MTUw" width="50px"/> </div> <div class="col-sm-4 col-xs-8 top_critic col-sm-push-13"> </div> <div class="col-sm-13 col-xs-24 col-sm-pull-4 critic_name"> <a class="unstyled bold articleLink" href="/critic/liam-lacey/">Liam Lacey</a> <br/> <a href="/source-2876"> <em class="subtle">Original Cin</em> </a> </div> </div> <div class="col-xs-16 review_container"> <div class="review_icon icon small fresh"></div> <div class="review_area"> <div class="review_date subtle small"> January 22, 2019</div> <div class="review_desc"> <div class="the_review"> The luminous black-and-white musical melodrama is the kind Hollywood used to make, about an attractive couple who sing and fight and love together. Through this lens, it's a kind of romantic protest against a p

In [97]:
review = reviews.findNext('div', {'class': 'row review_table_row'})

In [111]:
review = review.findNext('div', {'class': 'row review_table_row'})

In [135]:
critic = review.find('div', {'class': 'critic_name'}).find('a', {'class': 'articleLink'})
critic_name = critic.text.strip()
critic_url = critic.get('href')
date = review.find('div', {'class': 'review_date'}).text.strip()
text = review.find('div', {'class': 'the_review'}).text.strip()
rating = int(bool(review.find('div', {'class': 'fresh'})))
r = Review('movie_url', TYPE_CRITIC, critic_name, critic_url, rating, text, date)

Review(movie_url='movie_url', reviewer_type='critic', reviewer_name='Peter Canavese', reviewer_url='/critic/peter-canavese/', rating=1, text="A sweet romance this isn't, but Pawlikowski (Ida) balances the flatfooted realities of maddeningly thwarted love with swoony moments: smoky jazz clubs and songbird reveries.", date='January 23, 2019')


# USER REVIEWS

In [142]:
page_text = rq.get('https://www.rottentomatoes.com/m/cold_war_2018/reviews/?type=user').text

In [143]:
soup = BeautifulSoup(page_text, 'lxml')

In [144]:
reviews = soup.find('div', {'id': 'reviews'})

In [145]:
review = reviews.findNext('div', {'class': 'row review_table_row'})

In [146]:
review

<div class="row review_table_row"> <div class="col-xs-8"> <div class="col-sm-6 col-xs-15 critic_img"> <a class="" href="/user/id/906369511/"><img height="50" src="https://graph.facebook.com/v2.9/1793190188/picture" width="50"/></a></div> <div class="col-sm-7 col-xs-9 top_critic col-sm-push-13 superreviewer"> </div> <div class="col-sm-11 col-xs-24 col-sm-pull-4"> <a class="bold unstyled articleLink" href="/user/id/906369511/"><span style="word-wrap:break-word"> Kevin L</span> </a></div> </div> <div class="col-xs-16"> <span class="fl" style="color:#F1870A"> <span class="glyphicon glyphicon-star"></span><span class="glyphicon glyphicon-star"></span><span class="glyphicon glyphicon-star"></span><span class="glyphicon glyphicon-star"></span>½</span> <span class="fr small subtle">January 24, 2019</span> <div class="user_review" style="display:inline-block; width:100%"> <div class="scoreWrapper"><span class="45"></span></div> Bleak, beautiful, and awfully tragic, Cold War captures a rare type

In [147]:
review.find('div', {'class': 'col-sm-11'}).text.strip()

'Kevin L'

In [149]:
review.find('div', {'class': 'col-sm-11'}).find('a', {'class': 'articleLink'}).get('href')

'/user/id/906369511/'

In [150]:
review.find('div', {'class': 'user_review'}).text.strip()

'Bleak, beautiful, and awfully tragic, Cold War captures a rare type of romance that only foreign cinema can offer; the cinematography, subtle performances, and directing by Pawlikowski are spot-on'

# DB

In [206]:
conn = sqlite3.connect('/Users/jakubtlalka/it/ds/projects/movies/movies.db')

In [210]:
c = conn.cursor()

In [174]:
for url, in c.fetchall():
    print(url)

https://www.rottentomatoes.com/m/cold_war_2018


In [176]:
import dateparser

In [178]:
dateparser.parse(r.date).date()

datetime.date(2019, 1, 23)

In [179]:
r

Review(movie_url='movie_url', reviewer_type='critic', reviewer_name='Peter Canavese', reviewer_url='/critic/peter-canavese/', rating=1, text="A sweet romance this isn't, but Pawlikowski (Ida) balances the flatfooted realities of maddeningly thwarted love with swoony moments: smoky jazz clubs and songbird reveries.", date='January 23, 2019')

In [211]:
c.execute("INSERT INTO rt_reviews (movie_url, type, reviewer_name, reviewer_url, rating, review_text, review_date) VALUES ('{movie_url}', '{reviewer_type}', '{reviewer_name}', '{reviewer_url}', {rating}, '{parsed_text}', '{parsed_date}')".format(**r._asdict(), parsed_text=r.text.replace("'", "''"), parsed_date=str(dateparser.parse(r.date).date())))

In [194]:
c.execute("INSERT INTO rt_reviews (movie_url, type, reviewer_name, reviewer_url, rating, review_text, review_date) VALUES ('movie_url', 'critic', '/critic/peter-canavese/', '/critic/peter-canavese/', 1, 'A sweet romance this isn''t, but Pawlikowski (Ida) balances the flatfooted realities of maddeningly thwarted love with swoony moments: smoky jazz clubs and songbird reveries.', '2019-01-23')")

In [212]:
conn.commit()

In [19]:
import sqlalchemy as db

In [24]:
engine = db.create_engine('sqlite:////Users/jakubtlalka/it/ds/projects/movies/movies.sqlite')
conn = engine.connect()

In [25]:
metadata = db.MetaData()

In [227]:
rt_reviews = db.Table('rt_reviews', metadata, autoload=True, autoload_with=engine)

In [28]:
rt_reviews = db.Table('rt_reviews2', metadata,
                      db.Column('movie_url', db.TEXT()),
                      db.Column('type', db.TEXT()),
                      db.Column('reviewer_name', db.TEXT()),
                      db.Column('reviewer_url', db.TEXT()),
                      db.Column('rating', db.INTEGER()),
                      db.Column('review_text', db.TEXT()),
                      db.Column('review_date', db.TEXT()))

In [29]:
metadata.create_all(engine)

In [229]:
list(rt_reviews.columns)

[Column('movie_url', TEXT(), table=<rt_reviews>),
 Column('type', TEXT(), table=<rt_reviews>),
 Column('reviewer_name', TEXT(), table=<rt_reviews>),
 Column('reviewer_url', TEXT(), table=<rt_reviews>),
 Column('rating', INTEGER(), table=<rt_reviews>),
 Column('review_text', TEXT(), table=<rt_reviews>),
 Column('review_date', DATE(), table=<rt_reviews>)]

In [236]:
query = db.insert(rt_reviews).values(movie_url=r.movie_url,
                                     type=r.reviewer_type,
                                     reviewer_name=r.reviewer_name,
                                     reviewer_url=r.reviewer_url,
                                     rating=r.rating,
                                     review_text=r.text,
                                     review_date=dateparser.parse(r.date).date())

In [233]:
r

Review(movie_url='movie_url', reviewer_type='critic', reviewer_name='Peter Canavese', reviewer_url='/critic/peter-canavese/', rating=1, text="A sweet romance this isn't, but Pawlikowski (Ida) balances the flatfooted realities of maddeningly thwarted love with swoony moments: smoky jazz clubs and songbird reveries.", date='January 23, 2019')

In [237]:
str(query)

'INSERT INTO rt_reviews (movie_url, type, reviewer_name, reviewer_url, rating, review_text, review_date) VALUES (:movie_url, :type, :reviewer_name, :reviewer_url, :rating, :review_text, :review_date)'

In [241]:
conn.execute(query)

In [30]:
db.ARRAY?

In [31]:
db.Date

sqlalchemy.sql.sqltypes.Date

In [32]:
db.Text

sqlalchemy.sql.sqltypes.Text

In [33]:
class C:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def url(self):
        return f"{self.a} is a {self.b}"

In [34]:
C("life", "bitch").url()

'life is a bitch'